In [1]:
import cherrypy
import json
import redis
import uuid
from redis.commands.json.path import Path
from time import time

In [2]:
REDIS_HOST = 'redis-19577.c81.us-east-1-2.ec2.cloud.redislabs.com'
REDIS_PORT = 19577
REDIS_USERNAME = 'default'
REDIS_PASSWORD = 'so7BaeUMztM1pEpKusnIWrk9PN0nSVdm'

# Connect to Redis server
redis_client = redis.Redis(host=REDIS_HOST, port=REDIS_PORT, username=REDIS_USERNAME, password=REDIS_PASSWORD)
is_connected = redis_client.ping()
print('Redis Connected:', is_connected)


Redis Connected: True


In [3]:
# endpoint /status
class Devices(object):
    exposed = True


    def GET(self, *path, **query):
        reg = '*:battery'
        keys = redis_client.keys('*:battery')
        items = set()

        for key in keys:
            key = key.decode()
            mac = key.removesuffix(':battery')
            items.add(mac)
        


        response_dict = {
            'mac_addresses': list(items)
        }
        response = json.dumps(response_dict)

        cherrypy.response.status = "200 OK: Everything worked as expected"

        return response

In [4]:
#endpoint /device
class Device(object):
    exposed = True

    def GET(self, *path, **query):

        #Check if the path is inserted
        if len(path) != 1:
            #If not return a Bad request error
            raise cherrypy.HTTPError(400, message='Bad Request: missing MAC address.')

        mac_address = path[0]
        from_ = query.get('from', -1)
        to_ = query.get('to', -1)


        #Check if the start time is inserted
        if int(from_) < 0:
            #Return a Bad request error
            raise cherrypy.HTTPError(400, 'Bad Request: missing start time.')
        
        #Check if the end time is inserted
        if  int(to_) < 0:
            #Return a Bad request error
            raise cherrypy.HTTPError(400, 'Bad Request: missing end time.')

        
        battery_key = mac_address+':battery'
        power_key = mac_address+':power'
        print(type(battery_key))
        try:
            battery_values = redis_client.ts().range(battery_key, from_time=from_, to_time=to_)
            power_values = redis_client.ts().range(power_key, from_time=from_, to_time=to_)
        except:
            #Return a Bad request error
            raise cherrypy.HTTPError(404, 'Not Found: invalid MAC address.')
        
        
        response_dict = {
            'mac_address': mac_address,
            'timestamps': list(map(lambda x: x[0], battery_values)),
            'battery_levels': list(map(lambda x: x[1], battery_values)),
            'power_plugged':list(map(lambda x:x[1], power_values))
        }
        response = json.dumps(response_dict)


        cherrypy.response.status = "200 OK: Everything worked as expected"
        return response

    def DELETE(self, *path, **query):
        if len(path) != 1:
            raise cherrypy.HTTPError(400, 'Bad Request: missing MAC address')
        mac_address = path[0]
        #Try to remove the item
        battery_key = mac_address+':battery'
        power_key = mac_address+':power'
        print(battery_key)
        battery_found = redis_client.delete(battery_key)
        power_found = redis_client.delete(power_key)

        #Check if the item has been removed
        if battery_found == 0 and power_found == 0:
            raise cherrypy.HTTPError(404, 'Not Found: invalid MAC address')

        return 



In [5]:
conf = {'/': {'request.dispatch': cherrypy.dispatch.MethodDispatcher()}}
cherrypy.tree.mount(Devices(), '/devices', conf)
cherrypy.tree.mount(Device(), '/device', conf)
cherrypy.config.update({'server.socket_host': '0.0.0.0'})
cherrypy.config.update({'server.socket_port': 8080})
cherrypy.engine.start()
cherrypy.engine.block()

[27/Jan/2023:11:43:01] ENGINE Bus STARTING
[27/Jan/2023:11:43:01] ENGINE Started monitor thread 'Autoreloader'.
[27/Jan/2023:11:43:01] ENGINE Serving on http://0.0.0.0:8080
[27/Jan/2023:11:43:01] ENGINE Bus STARTED


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=81f45795-723f-4142-b302-f9037f9c1cf7' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>